In [170]:
from tools import analysistools as atools
import nanoparticle
from nanoparticle import Ligand, NanoParticle
import math
from membranesimulation import MembraneSimulation
import numpy as np
from tools import icosatiler
from tools import vectools
import k3d

In [166]:
def crossProd(u,v):
    return ((u[1]*v[2]-u[2]*v[1]),(u[2]*v[0]-u[0]*v[2]),(u[0]*v[1]-u[1]*v[0]))

In [2]:
def makeParticleModelFromPhenome(particle):
    points = []
    plot = k3d.plot()
    i = 1
    maxEps = 0
    for l in particle.ligands:
        if l.eps>maxEps:
            maxEps = l.eps
    for l in particle.ligands:
        if l.eps > 0.0:
            lx = l.rad*np.sin(l.polAng)*np.cos(l.aziAng)
            ly = l.rad*np.sin(l.polAng)*np.sin(l.aziAng)
            lz = l.rad*np.cos(l.polAng)
            lc = '0x' + ('#%02x%02x%02x' % (0, int((l.eps/maxEps)*255.0), 0))[1:]
            
            plot += k3d.points([lx,ly,lz],point_size=1,color=int(lc, 16))
            #plot += k3d.points([lx,ly,lz],point_size=1,color=int(0x00ffff*(l.eps/maxEps)))
            plot += k3d.text(str(i),[lx,ly,lz],color=0xffffff)
            i+=1
    plot += k3d.points([0,0,0],point_size=7,color=0xff0000)
    return plot

In [3]:
def buildCoveredParticle(eps,rad):
    particle = NanoParticle()
    lPos = icosatiler.cover72SpherePolar(rad)
    for v in lPos:
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [4]:
def spaceIsOccupied(particle,v):
    for l in particle.ligands:
        d = atools.greatArcDist((l.polAng,l.aziAng),(v[1],v[2]),particle.sig)
        if abs(d) < l.size:
            return True
    return False

In [106]:
def getClosestIcos(particle,v):
    lPos = icosatiler.cover72SpherePolar(particle.sig)
    closest = v
    minD = 1e8
    for l in lPos:
        v1 = icosatiler.sphPol2Crt(l) 
        v2 = icosatiler.sphPol2Crt(v)
        d = np.sum([j*j for j in np.subtract(v1,v2)])
        if abs(d) < minD:
            minD = d
            closest = l
    return closest

In [146]:
# lPos = icosatiler.cover72SpherePolar(4)
# dists = []
# minD = 1e8
# for v in lPos:
#     for l in lPos:
#         if l != v:
#             v1 = icosatiler.sphPol2Crt(l) 
#             v2 = icosatiler.sphPol2Crt(v)
#             d = np.sum([j*j for j in np.subtract(v1,v2)])
#             if abs(d) < minD:
#                 minD = d
#     dists.append(np.sqrt(minD))
# print np.mean(dists)

1.5990747678128467


In [52]:
def addLigand(particle,eps,v,lock=False):
    if lock:
        v = getClosestIcos(particle,v)
            
    if not spaceIsOccupied(particle,v):
        particle.addLigand(Ligand(eps,1,v[0],v[1],v[2]))
    return particle

In [53]:
def addBelt(particle,eps,rad,n,phi,lock=False):
    step = np.pi*2/float(n)
    for i in range(n):
        v = (rad,step*i,phi)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [54]:
def getArcAngle(dist,rad):
    return dist/rad

In [284]:
def addRing(particle,eps,rad,pos,n,spacing,lock=False):
    step = np.pi*2/float(n)
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    for i in range(n):
        tD = getArcAngle(spacing,rad)
        tP = step*i
        v = (rad,tD,tP)
        vC = icosatiler.sphPol2Crt(v)
        v1 = icosatiler.sphPol2Crt((rad,0,0))
        v2 = icosatiler.sphPol2Crt((rad,pos[0],pos[1]))
        v3 = crossProd(v1,v2)
        ang = np.arccos((np.dot(v1,v2))/(np.sqrt(np.sum([j*j for j in v1]))*np.sqrt(np.sum([j*j for j in v2]))))
        rM = vectools.buildERMatrix(v3, ang)
        vC = np.dot(rM,vC)
        v = icosatiler.crt2SphPol(vC)
        particle = addLigand(particle,eps,v,lock)
    return particle

In [285]:
def addPatch(particle,eps,rad,pos,n,spacing,lock=False):
    if lock:
        origin = getClosestIcos(particle,(rad,pos[0],pos[1]))
        pos = (origin[1],origin[2])
    particle = addLigand(particle,eps,(rad,pos[0],pos[1]),lock)
    particle = addRing(particle,eps,rad,pos,n,spacing)
    return particle

In [286]:
lPos = icosatiler.cover72SpherePolar(4)
print (lPos[0][1],lPos[0][2])

(1.2188879348723525, 0.7627921485173454)


In [305]:
p = NanoParticle()

In [306]:
p = addPatch(p,6,4,(1.0,0),10,1.6,lock=True)
# p = addRing(p,10,4,(lPos[0][1],lPos[0][2]),6,0,lock=False)

In [307]:
makeParticleModelFromPhenome(p)

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[120955991760, 120955838864, 120955698576, 120955839760, 120955697616, 120955990096, 120955990416, 120955991440, 120955990992, 120955993040, 120955993104, 120955992976, 120955990224])

In [141]:
makeParticleModelFromPhenome(buildCoveredParticle(10,4))

Plot(antialias=True, background_color=16777215, grid=[-1, -1, -1, 1, 1, 1], height=512, object_ids=[120951468368, 120951177680, 120951335312, 120951111504, 120951335184, 120951468688, 120951469904, 120951333136, 120951179344, 120951469712, 120951336784, 120951469456, 120951470736, 120951334800, 120951471312, 120951470288, 120951471632, 120951470416, 120951469584, 120951471696, 120951469648, 120951471120, 120951471824, 120951470608, 120951471376, 120951471760, 120951471952, 120951644752, 120951468496, 120951471440, 120951644624, 120951645648, 120951644880, 120951645072, 120951645520, 120951645008, 120951645968, 120951645264, 120951645392, 120951645840, 120951644496, 120951646224, 120951647632, 120951646608, 120951647760, 120951647568, 120951646480, 120951648080, 120951644560, 120951647952, 120951775440, 120951775376, 120951647888, 120951648208, 120951648016, 120951775952, 120951775888, 120951647504, 120951776016, 120951775504, 120951776464, 120951777744, 120951777808, 120951777552, 120951778448, 120951778000, 120951778768, 120951777232, 120951779024, 120951779088, 120951777296, 120951776208, 120951778256, 120951778384, 120951777936, 120951923728, 120951778512, 120951779152, 120951923664, 120951778896, 120951777488, 120951924304, 120951924176, 120951922768, 120951924240, 120951924752, 120951924560, 120951923472, 120951924368, 120951923856, 120951924624, 120951925328, 120951926352, 120951924816, 120951926672, 120951924944, 120951926544, 120951925648, 120951926032, 120951926160, 120952038288, 120951925392, 120951926736, 120952037520, 120952037776, 120952038480, 120952039248, 120952037456, 120952039568, 120952037584, 120952038544, 120952038608, 120952038160, 120952040208, 120952039440, 120952039632, 120952040848, 120951925136, 120952040720, 120952039504, 120952040144, 120952037648, 120952041040, 120952040912, 120952037904, 120952041168, 120952193360, 120952040080, 120952194384, 120952193296, 120952038800, 120952194576, 120952194192, 120952193488, 120952194704, 120952193424, 120952195664, 120952195216, 120952194896, 120952193808, 120952193936, 120952196176, 120952193744, 120952195408, 120951333520])